In [1]:
import requests
from datetime import datetime as dt
page_size = 100
today = int(dt.today().timestamp())


In [100]:
fields_list = ['id', 'title', 'label', 'date_from', 'date_to', 'kind',
              'free', 'text', 'restriction', 'status', 'organizations',
               'spheres', 'auditories', 'themes', 'areas',
               'districts', 'spot', 'place_type_id', 'place_types', 'foundation',
               'lon', 'lat', 'status', 'image'] # 
fields = ",".join(fields_list)
filter = '{}' # '{"<=occurrences.date_from":"2022-06-27+23:59:59",">=occurrences.date_from":"2020-05-27+00:00:00"}'
page_size = 2
page=1
sort = 'occurrences.date_to,-occurrences.date_from'
link = 'https://www.mos.ru/api/newsfeed/v4/frontend/json/ru/afisha?expand=place_types,spheres,themes,districts,areas,foundations,spots&fields={}&filter={}&per-page={}&sort={}&page={}'.format(
    fields, filter, page_size, sort, page)
r = requests.get(link)

In [101]:
def read_data(link, page=1):
    r = requests.get('{}&page={}'.format(link, page))
    if r.status_code == 200:
        if r.json()['_meta']['currentPage'] == r.json()['_meta']['pageCount']:
            return r.json()['items']
        return r.json()['items'] + read_data(link, page + 1)
    print('bad_status: {}'.format(r.status_code))
    return []
link = 'https://www.mos.ru/api/newsfeed/v4/frontend/json/ru/afisha?expand=spheres,themes,districts,areas,foundations,spots&fields={}&filter={}&per-page={}&sort={}'.format(
        fields, filter, page_size, sort)
result = read_data(link)

In [5]:
len(result)

1054

In [6]:
"""
+ title
+ text
+ status
+ image
- price_from (free)
- price_to
+ date_from
+ date_to
+ restriction
- id территор. признаков
+ id района
+ id учреждения / площадки
+ id тип мероприятия
+ id темы
"""

'\n+ title\n+ text\n+ status\n+ image\n- price_from (free)\n- price_to\n+ date_from\n+ date_to\n+ restriction\n- id территор. признаков\n+ id района\n+ id учреждения / площадки\n+ id тип мероприятия\n+ id темы\n'

In [7]:
result[0]

{'date_from': '2021-06-29 14:00:00',
 'date_to': '2021-06-29 21:30:00',
 'districts': [{'area_ids': [63501], 'id': 5500}],
 'free': 0,
 'id': 98016257,
 'kind': {'id': 'afisha', 'title': 'События афиши', 'type': 48},
 'label': None,
 'organizations': [12585090],
 'restriction': {'age': 12},
 'spheres': [{'activated': 1,
   'id': 80299,
   'priority': 0,
   'special': 0,
   'title': 'Спектакли'}],
 'spots': [{'address': 'Улица Сущевский Вал, дом 10а, строение 2',
   'created_at': '2021-04-23 15:13:17',
   'created_at_timestamp': 1619179997,
   'ebs_id': 273949,
   'ebs_title': 'Сущёвский Вал ул., 10А, строение 2',
   'foundation_id': 40211,
   'id': 213210,
   'lat': '55.79013',
   'lon': '37.594144',
   'place_type_id': 33085,
   'title': 'Улица Сущевский Вал, дом 10а, строение 2',
   'updated_at': '2021-05-20 19:26:30',
   'updated_at_timestamp': 1621527990}],
 'status': 'public',
 'text': '<p>Молодые актеры театра-студии &laquo;Три ступени&raquo; покажут спектакль &laquo;Прометей при

In [8]:
from google.colab import drive
drive.mount('/content/MyDrive/')

Mounted at /content/MyDrive/


In [11]:
import json
events = result

In [10]:
with open('MyDrive/MyDrive/data.json', 'w') as f:
    json.dump(result, f)

Нужны данные по:
- темы
- районы 
- сферы
-spots
-place_types

In [13]:
district = {1500: 'Цетральный',
            2500: 'Южный',
            3500: 'Северный', 
            4500: 'Юго-Западный',
            5500: 'Северо-Восточный',
            6500: 'Западный',
            7500: 'Восточный',
            8500: 'Северо-Восточный',
            9500: 'Юго-Восточный',
            10500: 'Зеленоградский',
            11500: 'Троицкий и Новомосковский',
            12500: 'Московская область'}

In [ ]:
link = 'https://www.mos.ru/api/newsfeed/v4/frontend/json/ru/afisha?expand=place_types,spheres,themes,districts,areas,foundations,spots&fields={}&filter={}&per-page={}&sort={}&page={}'.format(


In [66]:
link = 'https://www.mos.ru/api/newsfeed/v4/frontend/json'


In [62]:
def get_themes(link, lib, is_ru=True, page=1):
    suffix = ''
    if is_ru:
        suffix = '/ru'
    req = "{}{}/{}?page={}".format(link, suffix, lib, page)
    r = requests.get(req)
    if r.status_code == 200:
        if r.json()['_meta']['currentPage'] == r.json()['_meta']['pageCount']:
            return r.json()['items']
        return r.json()['items'] + get_themes(link, lib, is_ru=is_ru, page=page+1)
    return []

In [92]:
link = 'https://www.mos.ru/api/newsfeed/v4/frontend/json'

themes = get_themes(link, 'themes')
spheres = get_themes(link, 'spheres')
spots = get_themes(link, 'spot')

link = 'https://www.mos.ru/api/newsfeed/v4/frontend/json'
suffix = '/ru'
lib = 'afisha/place_types'
req = "{}{}/{}?page={}".format(link, suffix, lib, 1)
r = requests.get(req)
places = r.json()

link = 'https://www.mos.ru/api/directories/v2/frontend/json/territory'
areas = get_themes(link, 'areas', False)

for i in [areas, places, spots, spheres, themes, district, events]:
    print(len(i))

146
1
1968
56
71
12
1054


In [98]:
a = {'question': ['possible_answers', '1']}
list(a.values())[0][0]

'possible_answers'

{'items': [{'id': 32085, 'sort': 9000, 'title': 'ВДНХ'},
  {'id': 27085, 'sort': 13000, 'title': 'Библиотеки'},
  {'id': 1085, 'sort': 15000, 'title': 'Театры'},
  {'id': 2085, 'sort': 17000, 'title': 'Парки'},
  {'id': 3085, 'sort': 19000, 'title': 'Музеи'},
  {'id': 4085, 'sort': 21000, 'title': 'Выставочные залы'},
  {'id': 5085, 'sort': 23000, 'title': 'Концертные залы'},
  {'id': 6085, 'sort': 25000, 'title': 'Спортивные объекты'},
  {'id': 7085, 'sort': 27000, 'title': 'Образовательные объекты'},
  {'id': 8085,
   'sort': 29000,
   'title': 'Городское (общественное) пространство'},
  {'id': 9085, 'sort': 31000, 'title': 'Кинотеатры'},
  {'id': 10085, 'sort': 33000, 'title': 'Интернет'},
  {'id': 33085, 'sort': 34000, 'title': 'Культурные центры'},
  {'id': 31085, 'sort': 35000, 'title': 'Дом культуры'},
  {'id': 11085, 'sort': 36000, 'title': 'Катки'},
  {'id': 13085, 'sort': 38000, 'title': 'Другое'}]}

In [105]:
with open(os.path.join(path, 'events.json'), 'w') as f:
    json.dump(result, f)

In [95]:
path = 'MyDrive/MyDrive/mos.ru jsons'
import os
with open(os.path.join(path, 'areas.json'), 'w') as f:
    json.dump(areas, f)
with open(os.path.join(path, 'places.json'), 'w') as f:
    json.dump(places, f)
with open(os.path.join(path, 'spots.json'), 'w') as f:
    json.dump(spots, f)
with open(os.path.join(path, 'spheres.json'), 'w') as f:
    json.dump(spheres, f)
with open(os.path.join(path, 'themes.json'), 'w') as f:
    json.dump(themes, f)
with open(os.path.join(path, 'district.json'), 'w') as f:
    json.dump(district, f)
with open(os.path.join(path, 'events.json'), 'w') as f:
    json.dump(events, f)

In [86]:
link = 'https://www.mos.ru/api/newsfeed/v4/frontend/json/ru/spheres?per-page=50'
r = requests.get(link)


In [106]:
with open(os.path.join(path, 'events.json'), 'r') as f:
    res = json.load(f)

In [107]:
res[0]

{'date_from': '2021-06-29 14:00:00',
 'date_to': '2021-06-29 21:30:00',
 'districts': [{'area_ids': [63501], 'id': 5500}],
 'free': 0,
 'id': 98016257,
 'image': {'1x1_m': {'height': 800,
   'id': 3173004281,
   'src': '/upload/newsfeed/afisha/imagesWRpb1x(3)(7).jpg',
   'title': None,
   'type': '1x1_m',
   'width': 800},
  '1x1_s': {'height': 480,
   'id': 3173005281,
   'src': '/upload/newsfeed/afisha/imagesWRpb1x(3)(16).jpg',
   'title': None,
   'type': '1x1_s',
   'width': 480},
  '2x1_b': {'height': 800,
   'id': 3173006281,
   'src': '/upload/newsfeed/afisha/imagesWRpb1x(3)(3).jpg',
   'title': None,
   'type': '2x1_b',
   'width': 1600},
  '2x1_m': {'height': 400,
   'id': 3173007281,
   'src': '/upload/newsfeed/afisha/imagesWRpb1x(3)(2).jpg',
   'title': None,
   'type': '2x1_m',
   'width': 800},
  '2x1_s': {'height': 240,
   'id': 3173008281,
   'src': '/upload/newsfeed/afisha/imagesWRpb1x(3)(15).jpg',
   'title': None,
   'type': '2x1_s',
   'width': 480},
  '3x1_b': {'hei